<a href="https://colab.research.google.com/github/yohangumiel/Aulas-PUC-PR/blob/main/Agentes%20Conversacionais/Aula_2_chatbot_FAQ_banco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chatbot de FAQ bancário**.

## Adaptação voltada para responder alguns dos tópicos propostos no código disponível em: https://colab.research.google.com/drive/10XnyVjbKMftXEmpztTZZje-4HwtKBlrO

## Código desenvolvido por Prof. [Lucas Oliveira](https://www.linkedin.com/in/lucassilvaoliveira/) (PUC-PR) e Prof. [Yohan Gumiel](https://www.linkedin.com/in/yohan-gumiel-93b33b95/) (PUC-PR/UFMG)









## TAREFAS QUE SERÃO IMPLEMENTADAS DURANTE O CÓDIGO:


1.   Organizar melhor o chatbot ao utilizar uma função única para gerenciamento de diálogos, removendo, assim, as funções de controle de saudações, agradecimento e despedidas.
2.   Atualmente, o corpus de FAQ contém apenas as respostas possíveis. Poderíamos adicionar a ele as perguntas relativas a cada resposta, de modo a verificar a similaridade entre a pergunta feita pelo usuário e a pergunta do FAQ, e não com a possível resposta. Com isso, espera-se que o chatbot seja mais preciso ao encontrar respostas, pois a similaridade encontrada tenderá a ser maior.
3. Utilizar web scraping para montar corpus de FAQ automaticamente, ao ler todas as perguntas e respostas diretamente do site do banco


### Implementação utilizada - [Chatbot](https://github.com/kunkaweb/Chatbot)
Este chatbot é **híbrido**, que utiliza-se de regras e dados em um corpus. Ele faz uso de tecnicas básicas de Processamento de Linguagem Natural, implementadas nas bibliotecas **nltk** e **scikit-learn**.

In [1]:
import nltk
import numpy as np
import random
import string 
import warnings
import os
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt'); 
nltk.download('wordnet');
nltk.download('rslp')
nltk.download('stopwords')

# Define se imprimirá em tela informações de DEBUG
IS_DEBUG = False

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
!git clone https://github.com/kunkaweb/Chatbot.git

Cloning into 'Chatbot'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 25 (delta 1), reused 6 (delta 1), pack-reused 18
Unpacking objects: 100% (25/25), done.


#### Carregando corpus do FAQ
Aqui nós carregamos um corpus com as perguntas frequentes do banco, disponíveis em arquivo texto. Este documento pode ser editado a qualquer momento, seja para inserir novas questões, ou apagar e editar questões já existentes.

In [4]:
# Abre arquivo e coloca em minúsculas
f = open('Chatbot/faq-corpus.txt','r')
raw = f.read().lower()

# Segmenta sentenças e tokeniza o corpus
sent_tokens = nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

In [5]:
print('tamanho da lista: ', len(sent_tokens),'\n\n')
for item in sent_tokens:
    print(item,'\n')

tamanho da lista:  6 


você pode abrir sua conta em uma de nossas agências ou unidades do correspondente mais bb e banco postal. 

menores de 18 anos (não emancipados) deverão comparecer à agência acompanhados do responsável legal, munidos dos respectivos documentos. 

os documentos necessários são cpf, documento de identificação original com foto, comprovante de endereço e renda atualizados (até 90 dias), caso não possua comprovante de residência no próprio nome, é aceita declaração do titular do comprovante com firma reconhecida em cartório. 

é necessário que você e as pessoas que serão incluídas compareçam à agência, todos os titulares devem apresentar: cpf, documento de identificação original com foto, comprovantes de endereço e renda atualizados (ambos com validade de até 90 dias). 

sim, será necessário que todos assinem documento específico disponibilizado na agência que mantém a conta ou que estejam presentes e façam a concordância formal mediante carta própria assinada por t

#### Normalização e redução de dimensionalidade
Aqui definimos funções que utilizam o processo de Stemming nos textos, para normalizar termos similares a uma única forma. Exemplo: "gato", "gatos", "gata" e "gatas" são normalizados para "gat".

In [6]:
stemmer = nltk.stem.RSLPStemmer()

def LemTokens(tokens):
    return [stemmer.stem(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

LemNormalize('oi como você vai.!!! nada, então')

['oi', 'com', 'voc', 'vai', 'nad', 'ent']

In [7]:
LemNormalize('estudar estudo estudamos estudei')

['estud', 'estud', 'estud', 'estud']

## **Primeira tarefa**: Poderíamos organizar melhor nosso chatbot ao utilizar uma **função única para gerenciamento de diálogos**, assim poderíamos remover as funções de controle de saudações, agradecimento e despedidas.

#### **Resposta**: A função dialog mostrada abaixo gerencia todos os diálogos, e usa de expressões regulares para buscar ocorrências das saudações, agradecimentos e despedidas na string (entrada)

2.   Atualmente, o corpus de FAQ contém apenas as respostas possíveis. Poderíamos adicionar a ele as perguntas relativas a cada resposta, de modo a verificar a similaridade entre a pergunta feita pelo usuário e a pergunta do FAQ, e não com a possível resposta. Com isso, espera-se que o chatbot seja mais preciso ao encontrar respostas, pois a similaridade encontrada tenderá a ser maior.
3. Utilizar web scraping para montar corpus de FAQ automaticamente, ao ler todas as perguntas e respostas diretamente do site do banco



In [8]:
saudacoes = ("olá", "oi", "ei", "blz", "bom dia", "boa tarde", "boa noite")
agradecimentos = ("obrigado", "muito obrigado", "obrigada", "muito obrigada", "agradecido", "agradecida")
despedidas = ("tchau", "até logo", "fui", "adeus", "até mais")

respostas_bot = ("Oi", "Olá")

def dialog(sentence):
    saudacao_bool = False
    agradecimento_bool = False
    despedida_bool = False

    for saudacao in saudacoes:
        if re.findall(r'\b' + saudacao + r'\b', sentence):
            saudacao_bool = True
            break

    for agradecimento in agradecimentos:
        if re.findall(r'\b' + agradecimento + r'\b', sentence):
            agradecimento_bool = True
            break

    for despedida in despedidas:
        if re.findall(r'\b' + despedida + r'\b', sentence):
            despedida_bool = True
            break

    return saudacao_bool, agradecimento_bool, despedida_bool



In [9]:
dialog('oi tudo bem?')

(True, False, False)

In [10]:
dialog('oi obrigado tchau')

(True, True, True)

In [11]:
dialog('até mais, muito obrigado')

(False, True, True)

In [12]:
dialog('bom dia queria informações')

(True, False, False)

#### Controle de respostas
Definimos uma função para controlar as respostas dadas ao usuário. Para tal, utilizamos a métrica do **TF-IDF** e a **similaridade de cosseno**, para encontrar em nosso corpus a resposta que mais se encaixa na pergunta feita pelo usuário, ou seja, **a resposta cujo texto seja mais similar ao texto da pergunta**.

In [13]:
IS_DEBUG = False

def response(user_response):
    warnings.simplefilter('ignore')
    robo_response=''
    
    # Cria uma lista contendo todas sentenças do corpus + a sentença enviada pelo usuário. Assim geramos o TF-IDF de tudo.
    sentencas = sent_tokens + [user_response]
    # print(sentencas)

    # Inicia tranformação do texto em uma matriz TF-IDF - normaliza texto e retira stop-words
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=nltk.corpus.stopwords.words('portuguese'))

    tfidf = TfidfVec.fit_transform(sentencas)
    # Efetua cálculo de similaridade de cosseno entre o texto do usuário, e as sentenças do corpus
    vals = cosine_similarity(tfidf[-1], tfidf)
    # Obtém índice/posição da termo com maior grau de similaridade com o texto do usuário
    idx = vals.argsort()[0][-2]
    # Ordena vetor de similaridade
    flat = vals.flatten()
    flat.sort()
    # Obtém o valor de similaridade do penúltimo item no vetor
    # Pois o último item é o próprio texto do usuário, que obtém sempre grau máximo de similaridade
    # Portanto, obtemos a segunda maior similaridade, pois não queremos o próprio texto enviado pelo usuário
    req_tfidf = flat[-2]

    if IS_DEBUG:
      print("Vetor de similaridade (último valor é o próprio texto do usuário): ")
      print(vals)
      print("Índice da maior similaridade: ")
      print(idx)
      print("Vetor de similaridade ordenado: ")
      print(flat)
      print("Similaridade da sentença mais similar encontrada: ")
      print(req_tfidf)

    # Caso não seja encontrada similaridade para nenhum item do corpus
    if(req_tfidf==0):
        robo_response=robo_response+"Você poderia reformular sua pergunta?"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response


In [14]:
response('oi')

'Você poderia reformular sua pergunta?'

In [15]:
response('onde posso abrir minha conta do bb?')

'você pode abrir sua conta em uma de nossas agências ou unidades do correspondente mais bb e banco postal.'

In [16]:
response('quais documentos são necessários?')

'os documentos necessários são cpf, documento de identificação original com foto, comprovante de endereço e renda atualizados (até 90 dias), caso não possua comprovante de residência no próprio nome, é aceita declaração do titular do comprovante com firma reconhecida em cartório.'

#### Executando o chatbot
Trecho de código que controla a aplicação de maneira geral. A seguir algumas perguntas que você pode fazer ao chatbot:



1.   Como faço para abrir uma conta corrente?
2.   Quais são os documentos necessários?
3.   Do que devo tomar conhecimento?
4.   Quero incluir outra pessoa como titular em minha conta, o que devo fazer?


In [18]:
flag=True

print("FAQBOT: Olá, eu sou o faqbot. Posso responder suas perguntas referentes a abertura de conta no banco.")

while(flag==True):
    # Obtém entrada do usuário
    user_response = input()
    user_response=user_response.lower()

    saudacao, agradecimento, despedida = dialog(user_response)

    if despedida == False:
        if agradecimento == True:
            flag = False
            print("FAQBOT: Disponha. Estou aqui para lhe ajudar.")
        else:
            if saudacao == True:
                print("FAQBOT: " +  random.choice(respostas_bot))
            else:
                # Caso seja um pergunta válida
                print("FAQBOT: ",end="")
                print('\x1b[1;31m'+response(user_response)+'\x1b[0m')
    else:
        flag = False
        print("FAQBOT: Até mais!")


FAQBOT: Olá, eu sou o faqbot. Posso responder suas perguntas referentes a abertura de conta no banco.
olá
FAQBOT: Oi
tudo bem?
FAQBOT: Você poderia reformular sua pergunta?
como posso abrir uma conta?
FAQBOT: você pode abrir sua conta em uma de nossas agências ou unidades do correspondente mais bb e banco postal.
tchau
FAQBOT: Até mais!


## **Segunda tarefa**: Atualmente, o corpus de FAQ contém apenas as respostas possíveis. Poderíamos adicionar a ele as perguntas relativas a cada resposta, de modo a verificar a similaridade entre a pergunta feita pelo usuário e a pergunta do FAQ, e não com a possível resposta. Com isso, espera-se que o chatbot seja mais preciso ao encontrar respostas, pois a similaridade encontrada tenderá a ser maior.

#### **Resposta**: A função dialog mostrada abaixo gerencia todos os diálogos, e usa de expressões regulares para buscar ocorrências das saudações, agradecimentos e despedidas na string (entrada)




3. Utilizar web scraping para montar corpus de FAQ automaticamente, ao ler todas as perguntas e respostas diretamente do site do banco

In [19]:
import pandas as pd

In [20]:
!git clone https://github.com/yohangumiel/Aulas-PUC-PR

Cloning into 'Aulas-PUC-PR'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 44 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (44/44), done.


In [21]:
faq = pd.read_excel('Aulas-PUC-PR/Agentes Conversacionais/fag_perguntas_resposta.xlsx')

perguntas = faq['Pergunta'].values.tolist()
respostas = faq['Resposta'].values.tolist()

In [ ]:
faq.head()

,Pergunta,Resposta
0,Como faço para abrir uma conta corrente?,Você pode abrir sua conta em uma de nossas agê...
1,Quais são os principais documentos necessários...,"Os documentos necessários são CPF, documento d..."
2,Quero incluir outra pessoa como titular em min...,É necessário que você e as pessoas que serão i...
3,Posteriormente poderei excluir um desses titul...,Será necessário que todos assinem documento es...
4,Quais os cuidados que devo ter no momento da a...,Tomar conhecimento das tarifas cobradas pela a...


In [22]:
perguntas

['Como faço para abrir uma conta corrente?',
 'Quais são os principais documentos necessários para a abertura da conta?',
 'Quero incluir outra pessoa como titular em minha conta, o que devo fazer?',
 'Posteriormente poderei excluir um desses titulares?',
 'Quais os cuidados que devo ter no momento da abertura da conta?']

In [23]:
respostas

['Você pode abrir sua conta em uma de nossas agências ou unidades do Correspondente Mais BB e Banco Postal. Menores de 18 anos (não emancipados) deverão comparecer à agência acompanhados do responsável legal, munidos dos respectivos documentos.',
 'Os documentos necessários são CPF, documento de identificação original com foto, comprovante de endereço e renda atualizados (até 90 dias), caso não possua comprovante de residência no próprio nome, é aceita declaração do titular do comprovante com firma reconhecida em cartório.',
 'É necessário que você e as pessoas que serão incluídas compareçam à agência, Todos os titulares devem apresentar: CPF, documento de identificação original com foto, comprovantes de endereço e renda atualizados (ambos com validade de até 90 dias).',
 'Será necessário que todos assinem documento específico disponibilizado na agência que mantém a conta ou que estejam presentes e façam a concordância formal mediante carta própria assinada por todos.',
 'Tomar conheci

In [69]:
IS_DEBUG = False

def response_v1(user_response):
    warnings.simplefilter('ignore')
    robo_response=''
    
    # Cria uma lista contendo todas sentenças do corpus + a sentença enviada pelo usuário. Assim geramos o TF-IDF de tudo.
    sentencas = perguntas + [user_response]
    
    print('Perguntas disponíveis na base:\n')
    
    [print(sentenca) for sentenca in sentencas[:-1]]

    print('\n\n\n\n')
    # Inicia tranformação do texto em uma matriz TF-IDF - normaliza texto e retira stop-words
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=nltk.corpus.stopwords.words('portuguese'))

    tfidf = TfidfVec.fit_transform(sentencas)
    # Efetua cálculo de similaridade de cosseno entre o texto do usuário, e as sentenças do corpus
    vals = cosine_similarity(tfidf[-1], tfidf)
    # Obtém índice/posição da termo com maior grau de similaridade com o texto do usuário
    idx = vals.argsort()[0][-2]
    # Ordena vetor de similaridade
    flat = vals.flatten()
    flat.sort()
    # Obtém o valor de similaridade do penúltimo item no vetor
    # Pois o último item é o próprio texto do usuário, que obtém sempre grau máximo de similaridade
    # Portanto, obtemos a segunda maior similaridade, pois não queremos o próprio texto enviado pelo usuário
    req_tfidf = flat[-2]

    if IS_DEBUG:
      print("Vetor de similaridade (último valor é o próprio texto do usuário): ")
      print(vals)
      print("Índice da maior similaridade: ")
      print(idx)
      print("Vetor de similaridade ordenado: ")
      print(flat)
      print("Similaridade da sentença mais similar encontrada: ")
      print(req_tfidf)

    # Caso não seja encontrada similaridade para nenhum item do corpus
    if(req_tfidf==0):
        robo_response=robo_response+"Você poderia reformular sua pergunta?"
        return robo_response
    else:
        robo_response = robo_response+ 'PERGUNTA SELECIONADA: ' + perguntas[idx] + ' RESPOSTA SELECIONADA: ' + respostas[idx]
        return robo_response

In [70]:
response_v1('Abrir conta bb?')

Perguntas disponíveis na base:

Como faço para abrir uma conta corrente?
Quais são os principais documentos necessários para a abertura da conta?
Quero incluir outra pessoa como titular em minha conta, o que devo fazer?
Posteriormente poderei excluir um desses titulares?
Quais os cuidados que devo ter no momento da abertura da conta?







'PERGUNTA SELECIONADA: Como faço para abrir uma conta corrente? RESPOSTA SELECIONADA: Você pode abrir sua conta em uma de nossas agências ou unidades do Correspondente Mais BB e Banco Postal. Menores de 18 anos (não emancipados) deverão comparecer à agência acompanhados do responsável legal, munidos dos respectivos documentos.'

In [71]:
flag=True

print("FAQBOT: Olá, eu sou o faqbot. Posso responder suas perguntas referentes a abertura de conta no banco.")

while(flag==True):
    # Obtém entrada do usuário
    user_response = input()
    user_response=user_response.lower()

    saudacao, agradecimento, despedida = dialog(user_response)
    if despedida == False:
        if agradecimento == True:
            flag = False
            print("FAQBOT: Disponha. Estou aqui para lhe ajudar.")
        else:
            if saudacao == True:
                print("FAQBOT: " +  random.choice(respostas_bot))
            else:
                # Caso seja um pergunta válida
                print("FAQBOT: ",end="")
                print('\x1b[1;31m'+ response_v1(user_response) +'\x1b[0m')
    else:
        flag = False
        print("FAQBOT: Até mais!")

FAQBOT: Olá, eu sou o faqbot. Posso responder suas perguntas referentes a abertura de conta no banco.
olá
FAQBOT: Oi
tudo bem?
FAQBOT: Perguntas disponíveis na base:

Como faço para abrir uma conta corrente?
Quais são os principais documentos necessários para a abertura da conta?
Quero incluir outra pessoa como titular em minha conta, o que devo fazer?
Posteriormente poderei excluir um desses titulares?
Quais os cuidados que devo ter no momento da abertura da conta?





Você poderia reformular sua pergunta?
quero abrir uma conta?
FAQBOT: Perguntas disponíveis na base:

Como faço para abrir uma conta corrente?
Quais são os principais documentos necessários para a abertura da conta?
Quero incluir outra pessoa como titular em minha conta, o que devo fazer?
Posteriormente poderei excluir um desses titulares?
Quais os cuidados que devo ter no momento da abertura da conta?





PERGUNTA SELECIONADA: Como faço para abrir uma conta corrente? RESPOSTA SELECIONADA: Você pode abrir sua conta em 

## **Terceira tarefa**: Utilizar web scraping para montar corpus de FAQ automaticamente, ao ler todas as perguntas e respostas diretamente do site do banco.

#### **Resposta**: Código para scraping do FAQ do banco Itau, as perguntas "O que compõe o valor mínimo da minha fatura?" e "O que é pagamento mínimo da fatura?" levam para urls que tem formatação diferente então não foram obtidas as respostas para estas perguntas. As demais perguntas foram todas respondidas.

#### Link para FAQ do banco Itau: https://www.itau.com.br/atendimento-itau/para-voce/cartao-de-credito/


In [72]:
import requests
from bs4 import BeautifulSoup

In [73]:
import requests
from bs4 import BeautifulSoup   
import os
import pandas as pd


html_text = requests.get('https://www.itau.com.br/atendimento-itau/para-voce/cartao-de-credito/')
soup = BeautifulSoup(html_text.text, 'lxml')

lst_perguntas = []
divisoes = soup.find_all('div', class_ = 'accordion-item section')

for divisao in divisoes:
    topicos = divisao.find_all('a')    
    for topico in topicos:
        link = topico.get('href')
        pergunta = topico.text
        if pergunta != '':
            html_text2 = requests.get(link)
            soup2 = BeautifulSoup(html_text2.text, 'lxml')
            soup_text = soup2.find('div', class_ = 'richtext aem-GridColumn aem-GridColumn--default--12')
            
            resposta = ''
            if soup_text != None:
                for paragraph in soup_text.find_all('p'):
                    resposta += paragraph.text
        
            resposta = resposta.replace(u'\xa0', u' ')
            lst_perguntas.append([pergunta, resposta, link])
    

df_perguntas = pd.DataFrame(lst_perguntas, columns = ['Pergunta','Resposta','Link'])

In [74]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

df_perguntas.head(100)

,Pergunta,Resposta,Link
0,Onde consultar a fatura Itaucard e como saber ...,A data de fechamento da fatura varia a cada mê...,https://www.itau.com.br/atendimento-itau/para-...
1,Posso pagar conta com cartão de crédito?,"Sim, você pode pagar conta com cartão de crédi...",https://www.itau.com.br/atendimento-itau/para-...
2,O que é e como funciona o pagamento por aproxi...,“Aproxime e pague” é uma funcionalidade dispo...,https://www.itau.com.br/atendimento-itau/para-...
3,Como cadastrar ou cancelar o debito automático...,"Você, que tem conta-corrente no Itaú, pode cad...",https://www.itau.com.br/atendimento-itau/para-...
4,Como antecipar o pagamento de uma compra parce...,Você quer antecipar o pagamento das parcelas d...,https://www.itau.com.br/atendimento-itau/para-...
5,Onde posso pagar a fatura Itaú?,Pagar a fatura de cartão de credito é muito si...,https://www.itau.com.br/atendimento-itau/para-...
6,Como alterar a data de vencimento da fatura do...,Veja como alterar a data de vencimento da sua ...,https://www.itau.com.br/atendimento-itau/para-...
7,Como cadastrar a fatura digital?,"Conheça a fatura digital, saiba como cadastrar...",https://www.itau.com.br/atendimento-itau/para-...
8,Fiz um pagamentos em duplicidade como posso re...,Veja aqui como regularizar um pagamento em dup...,https://www.itau.com.br/atendimento-itau/para-...
9,O que é pagamento mínimo da fatura?,,https://www.itau.com.br/atendimento-itau/para-...


In [75]:
df_perguntas = df_perguntas[df_perguntas['Resposta'] != '']

In [76]:
df_perguntas

,Pergunta,Resposta,Link
0,Onde consultar a fatura Itaucard e como saber ...,A data de fechamento da fatura varia a cada mê...,https://www.itau.com.br/atendimento-itau/para-...
1,Posso pagar conta com cartão de crédito?,"Sim, você pode pagar conta com cartão de crédi...",https://www.itau.com.br/atendimento-itau/para-...
2,O que é e como funciona o pagamento por aproxi...,“Aproxime e pague” é uma funcionalidade dispo...,https://www.itau.com.br/atendimento-itau/para-...
3,Como cadastrar ou cancelar o debito automático...,"Você, que tem conta-corrente no Itaú, pode cad...",https://www.itau.com.br/atendimento-itau/para-...
4,Como antecipar o pagamento de uma compra parce...,Você quer antecipar o pagamento das parcelas d...,https://www.itau.com.br/atendimento-itau/para-...
5,Onde posso pagar a fatura Itaú?,Pagar a fatura de cartão de credito é muito si...,https://www.itau.com.br/atendimento-itau/para-...
6,Como alterar a data de vencimento da fatura do...,Veja como alterar a data de vencimento da sua ...,https://www.itau.com.br/atendimento-itau/para-...
7,Como cadastrar a fatura digital?,"Conheça a fatura digital, saiba como cadastrar...",https://www.itau.com.br/atendimento-itau/para-...
8,Fiz um pagamentos em duplicidade como posso re...,Veja aqui como regularizar um pagamento em dup...,https://www.itau.com.br/atendimento-itau/para-...
10,Como eu sei se meu cartão tem a funcionalidade...,Para saber se o seu cartão tem essa funcionali...,https://www.itau.com.br/atendimento-itau/para-...


In [77]:
perguntas_itau = df_perguntas['Pergunta'].values.tolist()
respostas_itau = df_perguntas['Resposta'].values.tolist()

In [78]:
perguntas_itau

['Onde consultar a fatura Itaucard e como saber se ela já fechou?',
 'Posso pagar conta com cartão de crédito?',
 'O que é e como funciona o pagamento por aproximação?',
 'Como cadastrar ou cancelar o debito automático da minha fatura?',
 'Como antecipar o pagamento de uma compra parcelada no cartão?',
 'Onde posso pagar a fatura Itaú?',
 'Como alterar a data de vencimento da fatura do cartão de crédito?',
 'Como cadastrar a fatura digital?',
 'Fiz um pagamentos em duplicidade como posso regularizar?',
 'Como eu sei se meu cartão tem a funcionalidade de pagamento por aproximação?',
 'Como saber o limite do cartão de crédito e o valor disponível para compra?',
 'Como aumento o limite do meu cartão de crédito?',
 'Quanto tempo leva para o limite de crédito ser liberado após o pagamento?',
 'O que é e como funciona o aumento de limite automático?',
 'O que é e como funciona o aumento de limite temporário do cartão de crédito?',
 'Como contrato ou cancelo a avaliação emergencial de crédito

In [79]:
respostas_itau

['A data de fechamento da fatura varia a cada mês, sendo geralmente 10 dias corridos antes do vencimento. Você consegue consultar a sua fatura Itaú através dos nossos canais digitais, ou até mesmo pelo WhatsApp, é só seguir o passo a passo aqui embaixo: - Pelo WhatsApp Itaucard:Escaneie o código QR ou clique aqui para conversar com o WhatsApp Itaucard. Se preferir, salve o número (11) 3003-3030 e chame a gente direto por lá. - Pelo App Itaucard:1. Abra o aplicativo Itaucard e faça login, com o número do seu cartão e senha.2. Na tela inicial toque no botão “ver fatura”.Agora você tem acesso a sua fatura em aberto e as dos meses anteriores. - Pelo Itaú na Internet:1. Acesse o site do Itaú e faça login. Se for correntista acesse com sua agência, conta corrente e senha eletrônica, mas caso só tenha o cartão Itaucard, acesse com o número dele e sua senha.2. Na página inicial, clique na aba “Cartões” e em seguida no cartão que deseja conferir a fatura.Pronto! Agora você tem acesso a sua fatu

In [86]:
IS_DEBUG = False

def response_itau(user_response):
    warnings.simplefilter('ignore')
    robo_response=''
    
    # Cria uma lista contendo todas sentenças do corpus + a sentença enviada pelo usuário. Assim geramos o TF-IDF de tudo.
    sentencas = perguntas_itau + [user_response]
    
    print('Perguntas disponíveis na base:\n')
    
    [print(sentenca) for sentenca in sentencas[:-1]]

    print('\n\n\n\n')
    # Inicia tranformação do texto em uma matriz TF-IDF - normaliza texto e retira stop-words
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=nltk.corpus.stopwords.words('portuguese'))

    tfidf = TfidfVec.fit_transform(sentencas)
    # Efetua cálculo de similaridade de cosseno entre o texto do usuário, e as sentenças do corpus
    vals = cosine_similarity(tfidf[-1], tfidf)
    # Obtém índice/posição da termo com maior grau de similaridade com o texto do usuário
    idx = vals.argsort()[0][-2]
    # Ordena vetor de similaridade
    flat = vals.flatten()
    flat.sort()
    # Obtém o valor de similaridade do penúltimo item no vetor
    # Pois o último item é o próprio texto do usuário, que obtém sempre grau máximo de similaridade
    # Portanto, obtemos a segunda maior similaridade, pois não queremos o próprio texto enviado pelo usuário
    req_tfidf = flat[-2]

    if IS_DEBUG:
      print("Vetor de similaridade (último valor é o próprio texto do usuário): ")
      print(vals)
      print("Índice da maior similaridade: ")
      print(idx)
      print("Vetor de similaridade ordenado: ")
      print(flat)
      print("Similaridade da sentença mais similar encontrada: ")
      print(req_tfidf)

    # Caso não seja encontrada similaridade para nenhum item do corpus
    if(req_tfidf==0):
        robo_response=robo_response+"Você poderia reformular sua pergunta?"
        return robo_response
    else:
        robo_response = robo_response+ 'PERGUNTA SELECIONADA: ' + perguntas_itau[idx] + ' RESPOSTA SELECIONADA: ' + respostas_itau[idx]
        return robo_response

In [87]:
response_itau('como posso abrir uma conta?')

Perguntas disponíveis na base:

Onde consultar a fatura Itaucard e como saber se ela já fechou?
Posso pagar conta com cartão de crédito?
O que é e como funciona o pagamento por aproximação?
Como cadastrar ou cancelar o debito automático da minha fatura?
Como antecipar o pagamento de uma compra parcelada no cartão?
Onde posso pagar a fatura Itaú?
Como alterar a data de vencimento da fatura do cartão de crédito?
Como cadastrar a fatura digital?
Fiz um pagamentos em duplicidade como posso regularizar?
Como eu sei se meu cartão tem a funcionalidade de pagamento por aproximação?
Como saber o limite do cartão de crédito e o valor disponível para compra?
Como aumento o limite do meu cartão de crédito?
Quanto tempo leva para o limite de crédito ser liberado após o pagamento?
O que é e como funciona o aumento de limite automático?
O que é e como funciona o aumento de limite temporário do cartão de crédito?
Como contrato ou cancelo a avaliação emergencial de crédito?
Como defino o limite de créd

'PERGUNTA SELECIONADA: Posso pagar conta com cartão de crédito? RESPOSTA SELECIONADA: Sim, você pode pagar conta com cartão de crédito e concentrar as suas contas na data de vencimento da sua fatura. Veja como é simples:É só acessar a sua conta-corrente, com agência e senha eletrônica, ou acessar o cartão, com número e senha do cartão de crédito, e digitar “pague contas” no campo de busca.Se preferir, você também pode pagar conta com cartão de crédito no guichê de qualquer agência Itaú. Para isso, é só levar o cartão de crédito, um documento com foto e a conta que deseja pagar.Importante: Para os cartões de crédito Hipercard, Sam’s Club Itaucard e Walmart Itaucard, o serviço está disponível exclusivamente pela central de atendimento a cartões e somente para contas de concessionárias (água, luz, telefone, gás e TV a cabo). Para o cartão Ipiranga Itaucard, o serviço só está disponível para cliente que possua conta-corrente no Itaú.Esse serviço está sujeito a encargos (juros e IOF), desde

In [88]:
response_itau('como posso pedir a segunda via do meu cartão itau?')

Perguntas disponíveis na base:

Onde consultar a fatura Itaucard e como saber se ela já fechou?
Posso pagar conta com cartão de crédito?
O que é e como funciona o pagamento por aproximação?
Como cadastrar ou cancelar o debito automático da minha fatura?
Como antecipar o pagamento de uma compra parcelada no cartão?
Onde posso pagar a fatura Itaú?
Como alterar a data de vencimento da fatura do cartão de crédito?
Como cadastrar a fatura digital?
Fiz um pagamentos em duplicidade como posso regularizar?
Como eu sei se meu cartão tem a funcionalidade de pagamento por aproximação?
Como saber o limite do cartão de crédito e o valor disponível para compra?
Como aumento o limite do meu cartão de crédito?
Quanto tempo leva para o limite de crédito ser liberado após o pagamento?
O que é e como funciona o aumento de limite automático?
O que é e como funciona o aumento de limite temporário do cartão de crédito?
Como contrato ou cancelo a avaliação emergencial de crédito?
Como defino o limite de créd

'PERGUNTA SELECIONADA: Como pedir a segunda via do cartão? RESPOSTA SELECIONADA: Precisando solicitar a segunda via do seu cartão? Eu te ajudo, é só seguir o passo a passo abaixo:- Cartão de crédito:Se você for correntista do Itaú, pode solicitar sua segunda via pelo aplicativo do Itaú.1.       Acesse sua conta com sua agência, conta e senha eletrônica.2.       Clique em serviços.3.       Em seguida, selecione cartões.4.      Toque em segunda via.Aí basta escolher o cartão e efetuar o pedido. Caso não seja correntista, você pode pedir através do aplicativo Itaucard, veja o passo a passo aqui embaixo:1.       Acesse com seu cartão e senha.\n2.       Toque em serviços.3.       Clique em pedir 2ª via do cartão.Aí basta escolher o cartão e efetuar o pedido.  - Cartão múltiplo (crédito e débito) e Cartão de débito: você pode solicitar sua segunda via pelo aplicativo do Itaú.1.       Acesse sua conta com sua agência, conta e senha eletrônica.2.       Clique na aba ajuda.3.       Procure pelo

In [89]:
response_itau('qual lugar eu pago fatura itau?')

Perguntas disponíveis na base:

Onde consultar a fatura Itaucard e como saber se ela já fechou?
Posso pagar conta com cartão de crédito?
O que é e como funciona o pagamento por aproximação?
Como cadastrar ou cancelar o debito automático da minha fatura?
Como antecipar o pagamento de uma compra parcelada no cartão?
Onde posso pagar a fatura Itaú?
Como alterar a data de vencimento da fatura do cartão de crédito?
Como cadastrar a fatura digital?
Fiz um pagamentos em duplicidade como posso regularizar?
Como eu sei se meu cartão tem a funcionalidade de pagamento por aproximação?
Como saber o limite do cartão de crédito e o valor disponível para compra?
Como aumento o limite do meu cartão de crédito?
Quanto tempo leva para o limite de crédito ser liberado após o pagamento?
O que é e como funciona o aumento de limite automático?
O que é e como funciona o aumento de limite temporário do cartão de crédito?
Como contrato ou cancelo a avaliação emergencial de crédito?
Como defino o limite de créd

'PERGUNTA SELECIONADA: Onde posso pagar a fatura Itaú? RESPOSTA SELECIONADA: Pagar a fatura de cartão de credito é muito simples. Mesmo sem ter conta no Itaú, você pode fazer o pagamento em nossos caixas eletrônicos e agências. Veja como: Facilite o seu dia a dia! Você que tem conta-corrente no Itaú pode cadastrar o débito automático para garantir o pagamento da fatura Itaú em dia, sem precisar se preocupar com a data de vencimento. Lembre-se: com o seu cartão Itaucard, você tem muitos benefícios.Além de 50% de desconto em diversos teatros e cinemas, você pode contratar o programa de pontos. Com ele, as suas compras viram pontos que podem ser trocados por descontos, prêmios e viagens.E não se esqueça: você também pode fazer compras online com descontos.É só conferir os parceiros participantes ;-)'

In [90]:
response_itau('o que essa tal de anuidade?')

Perguntas disponíveis na base:

Onde consultar a fatura Itaucard e como saber se ela já fechou?
Posso pagar conta com cartão de crédito?
O que é e como funciona o pagamento por aproximação?
Como cadastrar ou cancelar o debito automático da minha fatura?
Como antecipar o pagamento de uma compra parcelada no cartão?
Onde posso pagar a fatura Itaú?
Como alterar a data de vencimento da fatura do cartão de crédito?
Como cadastrar a fatura digital?
Fiz um pagamentos em duplicidade como posso regularizar?
Como eu sei se meu cartão tem a funcionalidade de pagamento por aproximação?
Como saber o limite do cartão de crédito e o valor disponível para compra?
Como aumento o limite do meu cartão de crédito?
Quanto tempo leva para o limite de crédito ser liberado após o pagamento?
O que é e como funciona o aumento de limite automático?
O que é e como funciona o aumento de limite temporário do cartão de crédito?
Como contrato ou cancelo a avaliação emergencial de crédito?
Como defino o limite de créd

'PERGUNTA SELECIONADA: O que é anuidade do cartão? RESPOSTA SELECIONADA: A anuidade do cartão é uma tarifa cobrada pela prestação de serviços como administração, manutenção, gerenciamento e monitoramento do cartão.Na Itaucard, o valor da anuidade do cartão volta para você em benefícios como:Confira os detalhes dos benefícios feitos para você ;-)Para conferir os valores de anuidade, aqui no site, você pode clicar em Produtos > Cartões > Tabela de Tarifas.Para outras informações da anuidade Itaucard, é só acessar a sua conta-corrente, com agência, conta e senha eletrônica, ou acessar o cartão, com número e senha do cartão de crédito, e digitar “atendimento online” no campo de busca.Se preferir, você pode acessar o aplicativo Itaucard e tocar na opção Atendimento > Chat.Se você não tem os dados para acessar o seu cartão pela internet, é só ligar para a central de atendimento a cartões, que tiramos as suas dúvidas por lá.'

In [91]:
flag=True

print("FAQBOT: Olá, eu sou o faqbot. Posso responder suas perguntas referentes a abertura de conta no banco.")

while(flag==True):
    # Obtém entrada do usuário
    user_response = input()
    user_response=user_response.lower()

    saudacao, agradecimento, despedida = dialog(user_response)
    if despedida == False:
        if agradecimento == True:
            flag = False
            print("FAQBOT: Disponha. Estou aqui para lhe ajudar.")
        else:
            if saudacao == True:
                print("FAQBOT: " +  random.choice(respostas_bot))
            else:
                # Caso seja um pergunta válida
                print("FAQBOT: ",end="")
                print('\x1b[1;31m'+ response_itau(user_response) +'\x1b[0m')
    else:
        flag = False
        print("FAQBOT: Até mais!")

FAQBOT: Olá, eu sou o faqbot. Posso responder suas perguntas referentes a abertura de conta no banco.
olá
FAQBOT: Oi
o que é essa tal de anuidade?
FAQBOT: Perguntas disponíveis na base:

Onde consultar a fatura Itaucard e como saber se ela já fechou?
Posso pagar conta com cartão de crédito?
O que é e como funciona o pagamento por aproximação?
Como cadastrar ou cancelar o debito automático da minha fatura?
Como antecipar o pagamento de uma compra parcelada no cartão?
Onde posso pagar a fatura Itaú?
Como alterar a data de vencimento da fatura do cartão de crédito?
Como cadastrar a fatura digital?
Fiz um pagamentos em duplicidade como posso regularizar?
Como eu sei se meu cartão tem a funcionalidade de pagamento por aproximação?
Como saber o limite do cartão de crédito e o valor disponível para compra?
Como aumento o limite do meu cartão de crédito?
Quanto tempo leva para o limite de crédito ser liberado após o pagamento?
O que é e como funciona o aumento de limite automático?
O que é e c

## Referências e Material complementar

* [Chatbot simples com NLTK e scikit-learn](https://github.com/kunkaweb/Chatbot)
* [scikit-learn - TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
* [scikit-learn - cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html)